# Глубинное обучение для текстовых данных, ФКН ВШЭ

## Домашнее задание 1: Text Suggestion

### Оценивание и штрафы

Максимально допустимая оценка за работу — 10 баллов. Сдавать задание после жесткого дедлайна нельзя. При сдачи решения после мягкого дедлайна за каждый день просрочки снимается по одному баллу.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Весь код должен быть написан самостоятельно. Чужим кодом для пользоваться запрещается даже с указанием ссылки на источник. В разумных рамках, конечно. Взять пару очевидных строчек кода для реализации какого-то небольшого функционала можно.

Неэффективная реализация кода может негативно отразиться на оценке. Также оценка может быть снижена за плохо читаемый код. Все ответы должны сопровождаться кодом или комментариями о том, как они были получены.

__Мягкий дедлайн: 1 дек

__Жесткий дедлайн: 4 дек


### О задании

В этом задании вам предстоит реализовать систему, предлагающую удачное продолжение слова или нескольких следующих слов в режиме реального времени по типу тех, которые используются в телефонах, поисковой строке или приложении почты. Полученную систему вам нужно будет обернуть в пользовательский интерфейс с помощью библиотеки [reflex](https://github.com/reflex-dev/reflex), чтобы ей можно было удобно пользоваться, а так же, чтобы убедиться, что все работает как надо. В этот раз вам не придется обучать никаких моделей, мы ограничимся n-граммной генерацией.

### Структура

Это домашнее задание состоит из двух частей предположительно одинаковых по сложности. В первой вам нужно будет выполнить 5 заданий, по итогам которых вы получите минимально рабочее решение. А во второй, пользуясь тем, что вы уже сделали реализовать полноценную систему подсказки текста с пользовательским интерфейсом. Во второй части мы никак не будем ограничивать вашу фантазию. Делайте что угодно, лишь бы получилось в результате получился удобный фреймворк. Чем лучше у вас будет результат, тем больше баллов вы получите. Если будет совсем хорошо, то мы добавим бонусов сверху по своему усмотрению.

### Оценивание
При сдаче зададания в anytask вам будет необходимо сдать весь код, а также отчет с подробным описанием техник, которые в применили для создания вашей системы. Не лишним будет также написать и о том, что у вас не получилось и почему.

За часть с заданиями можно будет получить до __5__ баллов, за отчет – до __3__ баллов, 2 балл за доп вопросы, если возникнут, если вопросов не возникло, считаем, что 2 балла вы получили 

## Часть 1

### Данные

Для получения текстовых статистик используйте датасет `emails.csv`. Вы можете найти его по [ссылке](https://disk.yandex.ru/d/ikyUhWPlvfXxCg). Он содержит более 500 тысяч электронных писем на английском языке.

In [1]:
import pandas as pd
import numpy as np
import re
import email
import time

from IPython.display import clear_output
from collections import defaultdict, Counter
from typing import Union, List

In [2]:
emails = pd.read_csv('emails.csv')
len(emails)

517401

In [3]:
emails.sample(15)

,file,message
400944,schoolcraft-d/deleted_items/639.,Message-ID: <27404034.1075860743039.JavaMail.e...
124938,germany-c/all_documents/1201.,Message-ID: <671277.1075853689632.JavaMail.eva...
268219,lay-k/all_documents/113.,Message-ID: <28236766.1075840203510.JavaMail.e...
261117,kitchen-l/sent_items/1678.,Message-ID: <25262822.1075855292090.JavaMail.e...
368594,rapp-b/inbox/134.,Message-ID: <1569665.1075859076389.JavaMail.ev...
396743,sanders-r/metals/3.,Message-ID: <140925.1075853240912.JavaMail.eva...
221631,kaminski-v/sent_items/1255.,Message-ID: <15856294.1075862456029.JavaMail.e...
228348,kean-s/all_documents/1819.,Message-ID: <7124714.1075846189168.JavaMail.ev...
12595,bass-e/all_documents/280.,Message-ID: <7395119.1075854581820.JavaMail.ev...
115616,fossum-d/_sent_mail/809.,Message-ID: <21098492.1075842566220.JavaMail.e...


In [4]:
emails.iloc[1][1]

"Message-ID: <15464986.1075855378456.JavaMail.evans@thyme>\nDate: Fri, 4 May 2001 13:51:00 -0700 (PDT)\nFrom: phillip.allen@enron.com\nTo: john.lavorato@enron.com\nSubject: Re:\nMime-Version: 1.0\nContent-Type: text/plain; charset=us-ascii\nContent-Transfer-Encoding: 7bit\nX-From: Phillip K Allen\nX-To: John J Lavorato <John J Lavorato/ENRON@enronXgate@ENRON>\nX-cc: \nX-bcc: \nX-Folder: \\Phillip_Allen_Jan2002_1\\Allen, Phillip K.\\'Sent Mail\nX-Origin: Allen-P\nX-FileName: pallen (Non-Privileged).pst\n\nTraveling to have a business meeting takes the fun out of the trip.  Especially if you have to prepare a presentation.  I would suggest holding the business plan meetings here then take a trip without any formal business meetings.  I would even try and get some honest opinions on whether a trip is even desired or necessary.\n\nAs far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what 

Заметьте, что данные очень грязные. В каждом письме содержится различная мета-информация, которая будет только мешать при предсказании продолжения текста.

__Задание 1 (1 балл).__ Очистите корпус текстов по вашему усмотрению. В идеале обработанные тексты должны содержать только текст самого письма и ничего лишнего по типу ссылок, адресатов и прочих символов, которыми мы точно не хотим продолжать текст. Оценка будет выставляться по близости вашего результата к этому идеалу.

In [5]:
def extract_cleaned_bodies(messages):
    cleaned_messages = []
    
    for msg in messages:
        try:
            # преобразуем сообщение в объект email
            email_obj = email.message_from_string(msg)
            raw_msg = email_obj.get_payload()  # извлекаем само тело письма
            
            # удаляем метаинформацию (заголовки, пересылки)
            clean_msg = re.split(r'^\s*[-]+ Forwarded by.*$', raw_msg, flags=re.MULTILINE)[-1]
            
            # удаляем заголовки (From, To, Date и т.д.)
            clean_msg = re.sub(r"(?im)^(from:|to:|sent:|subject:|date:|cc:|bcc:|x-.*:).*", "", clean_msg)
            
            # удаляем временные отметки и избыточные фрагменты заголовков в начале текста
            clean_msg = re.sub(r"(?im)^\s*\d{1,2}:\d{2}\s*(AM|PM)\s*.*\n", " ", clean_msg)
            
            # убираем текстовые метки времени и пересылок, остающиеся в середине письма
            clean_msg = re.sub(r"(?im)^\d{4,8}\s+(AM|PM)\s*.*$", "", clean_msg, flags=re.MULTILINE)
            
            # очищаем от специальных символов (оставляем буквы, цифры, пробелы, точки и запятые)
            clean_msg = re.sub(r'[^A-Za-z0-9 ,.\n]', '', clean_msg)
            
            # убираем табуляции и множественные переводы строк
            clean_msg = re.sub(r'\t', ' ', clean_msg)
            clean_msg = re.sub(r'\n+', ' ', clean_msg)
            
            # удаляем лишние пробелы
            clean_msg = re.sub(r' {2,}', ' ', clean_msg).strip()
            
            cleaned_messages.append(clean_msg)
        
        except Exception as e:
            # если что-то пошло не так, выводим сообщение об ошибке
            cleaned_messages.append(f"Error processing message: {e}")
    
    return cleaned_messages

In [6]:
%%time 

messages_body = extract_cleaned_bodies(emails['message'])
emails['body'] = messages_body
emails.head(5)

Wall time: 4min 53s


,file,message,body
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,Here is our forecast
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,Traveling to have a business meeting takes the...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,test successful. way to go
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,"Randy, Can you send me a schedule of the salar..."
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,Lets shoot for Tuesday at 1145.


In [7]:
emails.iloc[1][2] # посмотрим на результат обработки

'Traveling to have a business meeting takes the fun out of the trip. Especially if you have to prepare a presentation. I would suggest holding the business plan meetings here then take a trip without any formal business meetings. I would even try and get some honest opinions on whether a trip is even desired or necessary. As far as the business meetings, I think it would be more productive to try and stimulate discussions across the different groups about what is working and what is not. Too often the presenter speaks and the others are quiet just waiting for their turn. The meetings might be better if held in a round table discussion format. My suggestion for where to go is Austin. Play golf and rent a ski boat and jet skis. Flying somewhere takes too much time.'

Для следующего задания вам нужно будет токенизировать текст. Для этого просто разбейте его по словам. Очевидно, итоговый результат будет лучше, если ваша система также будет предлагать уместную пунктуацию. Но если вы считаете, что результат получается лучше без нее, то можете удалить все небуквенные символы на этапе токенизации.

In [8]:
def text_tokenization(text):
    text = text.lower() # приводим сначала текст к одному регистру
    
    tokens = re.findall(r'\w+|[^\w\s]', text) # будем разбивать текст на слова и сохраним пунктуацию
    return tokens

In [9]:
%%time
emails_token = [text_tokenization(em) for em in emails['body']]

Wall time: 7min 14s


In [10]:
emails_token[1] # помсотрим на нашем примере, как разбилось

['traveling',
 'to',
 'have',
 'a',
 'business',
 'meeting',
 'takes',
 'the',
 'fun',
 'out',
 'of',
 'the',
 'trip',
 '.',
 'especially',
 'if',
 'you',
 'have',
 'to',
 'prepare',
 'a',
 'presentation',
 '.',
 'i',
 'would',
 'suggest',
 'holding',
 'the',
 'business',
 'plan',
 'meetings',
 'here',
 'then',
 'take',
 'a',
 'trip',
 'without',
 'any',
 'formal',
 'business',
 'meetings',
 '.',
 'i',
 'would',
 'even',
 'try',
 'and',
 'get',
 'some',
 'honest',
 'opinions',
 'on',
 'whether',
 'a',
 'trip',
 'is',
 'even',
 'desired',
 'or',
 'necessary',
 '.',
 'as',
 'far',
 'as',
 'the',
 'business',
 'meetings',
 ',',
 'i',
 'think',
 'it',
 'would',
 'be',
 'more',
 'productive',
 'to',
 'try',
 'and',
 'stimulate',
 'discussions',
 'across',
 'the',
 'different',
 'groups',
 'about',
 'what',
 'is',
 'working',
 'and',
 'what',
 'is',
 'not',
 '.',
 'too',
 'often',
 'the',
 'presenter',
 'speaks',
 'and',
 'the',
 'others',
 'are',
 'quiet',
 'just',
 'waiting',
 'for',
 'the

## Дополнение слова

Описанная система будет состоять из двух частей: дополнение слова до целого и генерация продолжения текста (или вариантов продолжений). Начнем с первой части.

В этой части вам предстоит реализовать метод дополнения слова до целого по его началу (префиксу). Для этого сперва необходимо научиться находить все слова, имеющие определенный префикс. Мы будем вызывать функцию поиска подходящих слов после каждой напечатанной пользователем буквы. Поэтому нам очень важно, чтобы поиск работал как можно быстрее. Простой перебор всех слов занимает $O(|V| \cdot n)$ времени, где $|V|$ – размер словаря, а $n$ – длина префикса. Мы же напишем [префиксное дерево](https://ru.wikipedia.org/wiki/Префиксное_дерево), которое позволяет искать слова за $O(n + m)$, где $m$ – число подходящих слов.

__Задание 2 (1 балл).__ Допишите префиксное дерево для поиска слов по префиксу. Ваше дерево должно работать за $O(n + m)$ операции, в противном случае вы не получите баллов за это задание.

In [17]:
class PrefixTreeNode:
    def __init__(self):
        # словарь с буквами, которые могут идти после данной вершины
        self.children: dict[str, PrefixTreeNode] = {}
        self.is_end_of_word = False

class PrefixTree:
    def __init__(self, vocabulary: List[str]):
        """
        vocabulary: список всех уникальных токенов в корпусе
        """
        self.root = PrefixTreeNode()  # корень дерева
        for word in vocabulary:
            self.add_word(word)  # добавляем каждое слово в дерево

    def add_word(self, word: str):
        current = self.root  # начинаем с корня
        for w in word:
            if w not in current.children:
                current.children[w] = PrefixTreeNode()  # создаем новый узел, если его нет
            current = current.children[w]
        current.is_end_of_word = True  # отмечаем конец слова

    def search_prefix(self, prefix: str) -> List[str]:
        """
        Возвращает все слова, начинающиеся на prefix
        prefix: str – префикс слова
        """
        # рекурсивная функция для поиска всех листьев
        def _find_all_leaves(node, current_word, result):
            if node.is_end_of_word:
                result.append(current_word)

            for c, child in node.children.items():
                _find_all_leaves(child, current_word + c, result)

        # ищем узел, соответствующий последнему символу префикса
        start_node = self.root
        for p in prefix:
            if p not in start_node.children:
                return []  # значит префикс не найден
            start_node = start_node.children[p]

        # собираем все слова, начиная с узла start_node
        result = []
        _find_all_leaves(start_node, prefix, result)
        return result

In [18]:
vocabulary = ['aa', 'aaa', 'abb', 'bba', 'bbb', 'bcd']
prefix_tree = PrefixTree(vocabulary)

assert set(prefix_tree.search_prefix('a')) == set(['aa', 'aaa', 'abb'])
assert set(prefix_tree.search_prefix('bb')) == set(['bba', 'bbb'])

Теперь, когда у нас есть способ быстро находить все слова с определенным префиксом, нам нужно их упорядочить по вероятности, чтобы выбирать лучшее. Будем оценивать вероятность слова по частоте его встречаемости в корпусе.

__Задание 3 (1 балл).__ Допишите класс `WordCompletor`, который формирует словарь и префиксное дерево, а так же умеет находить все возможные продолжения слова вместе с их вероятностями. В этом классе вы можете при необходимости дополнительно отфильтровать слова, например, удалив все самые редкие. Постарайтесь максимально оптимизировать ваш код.

In [22]:
class WordCompletor:
    def __init__(self, corpus):
        """
        corpus: list – корпус текстов
        """
        self.word_counts = defaultdict(int)
        self.word_counter = 0
        
        # считаем частоты слов и общее количество слов
        for text in corpus:
            for word in text:
                self.word_counts[word] += 1
                self.word_counter += 1

        # формируем словарь уникальных слов
        vocabulary_words = list(self.word_counts.keys())
        # ну и в итоге создаем префиксное дерево
        self.prefix_tree = PrefixTree(vocabulary_words)

    def get_words_and_probs(self, prefix: str) -> (List[str], List[float]):
        """
        Возвращает список слов, начинающихся на prefix,
        с их вероятностями (нормировать ничего не нужно)
        """
        words, probs = [], []
        words = self.prefix_tree.search_prefix(prefix)
        probs = [self.word_counts[word] / self.word_counter for word in words]
        return words, probs

In [23]:
dummy_corpus = [
    ["aa", "ab"],
    ["aaa", "abab"],
    ["abb", "aa", "ab", "bba", "bbb", "bcd"],
]

word_completor = WordCompletor(dummy_corpus)
words, probs = word_completor.get_words_and_probs('a')
words_probs = list(zip(words, probs))
assert set(words_probs) == {('aa', 0.2), ('ab', 0.2), ('aaa', 0.1), ('abab', 0.1), ('abb', 0.1)}

## Предсказание следующих слов

Теперь, когда мы умеем дописывать слово за пользователем, мы можем пойти дальше и предожить ему несколько следующих слов с учетом дописанного. Для этого мы воспользуемся n-граммами и будем советовать n следующих слов. Но сперва нужно получить n-граммную модель.

Напомним, что вероятность последовательности для такой модели записывается по формуле
$$
P(w_1, \dots, w_T) = \prod_{i=1}^T P(w_i \mid w_{i-1}, \dots, w_{i-n}).
$$

Тогда, нам нужно оценить $P(w_i \mid w_{i-1}, \dots, w_{i-n})$ по частоте встречаемости n-граммы.   

__Задание 4 (1 балл).__ Напишите класс для n-граммной модели. Понятное дело, никакого сглаживания добавлять не надо, мы же не хотим, чтобы модель советовала случайные слова (хоть и очень редко).

In [27]:
class NGramLanguageModel:
    def __init__(self, corpus: List[List[str]], n: int):
        self.n = n
        self.count_ngrams = defaultdict(int)  # сначала сдеалем словарь для подсчета n-грамм
        self.count_context = defaultdict(int)  # похожий создадим и для подсчета контекстов

        # перейдем к созданию модели n-грамм:
        for c in corpus:
            sentence_len = len(c)
            for i in range(sentence_len):
                for j in range(1, sentence_len - i + 1):  # перебираем все возможные длины n-грамм
                    ngram = tuple(c[i:i + j])  # создаем n-грамму
                    self.count_ngrams[ngram] += 1
                    if len(ngram) > 1:  # контекст существует только для n-грамм длиной > 1
                        context = ngram[:-1]  # контекстом будут считаться все элементы, кроме последнего
                        self.count_context[context] += 1

    def get_next_words_and_probs(self, prefix: List[str]) -> (List[str], List[float]):
        """
        Возвращает список слов, которые могут идти после префикса,
        а также их вероятности.
        """
        prefix_tuple = tuple(prefix)  # преобразуем префикс в кортеж для поиска в словарях
        total_context_count = self.count_context.get(prefix_tuple, 0)

        # если контекст не найден, то возвращаем пустые списки
        if total_context_count == 0:
            return [], []

        # собираем возможные продолжения для следующего слова
        continuation_candidates = {
            ngram[-1]: self.count_ngrams[ngram]
            for ngram in self.count_ngrams
            if len(ngram) == len(prefix_tuple) + 1 and ngram[:-1] == prefix_tuple}

        # рассчитываем вероятности для каждого следующего слова
        next_words = list(continuation_candidates.keys())
        probabilities = [count / total_context_count for count in continuation_candidates.values()]

        return next_words, probabilities

In [28]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)

next_words, probs = n_gram_model.get_next_words_and_probs(['aa', 'aa'])
words_probs = list(zip(next_words, probs))

assert set(words_probs) == {('aa', 2/3), ('ab', 1/3)}

Отлично, мы теперь можем объединить два метода в автоматический дописыватель текстов: первый будет дополнять слово, а второй – предлагать продолжения. Хочется, чтобы предлагался список возможных продолжений, из который пользователь сможет выбрать наиболее подходящее. Самое сложное тут – аккуратно выбирать, что показывать, а что нет.   

__Задание 5 (1 балл).__ В качестве первого подхода к снаряду реализуйте метод, возвращающий всегда самое вероятное продолжение жадным способом. Если вы справитесь, то сможете можете добавить опцию поддержки нескольких вариантов продолжений, что сделает метод гораздо лучше.

In [33]:
class TextSuggestion:
    def __init__(self, word_completor, n_gram_model):
        self.word_completor = word_completor
        self.n_gram_model = n_gram_model

    def suggest_text(self, text: Union[str, list], n_words=3, n_texts=1) -> list[list[str]]:
        """
        Возвращает возможные варианты продолжения текста (по умолчанию только один)
        
        text: строка или список слов – написанный пользователем текст
        n_words: число слов, которые дописывает n-граммная модель
        n_texts: число возвращаемых продолжений (пока что только одно)
        
        return: list[list[str]] – список из n_texts списков слов, по 1 + n_words слов в каждом
        Первое слово – это то, которое WordCompletor дополнил до целого.
        """

        suggestions = []

        # преобразуем входной текст в список слов
        words = text.strip().split() if isinstance(text, str) else text[:]
        
        if not words:
            return []

        # после этого получаем дополнение к последнему слову
        last_word = words[-1]
        complets, probabilities = self.word_completor.get_words_and_probs(last_word)
        
        completed_word = complets[probabilities.index(max(probabilities))] if complets else last_word
        
        # далее составляем полный список слов для генерации продолжений
        updated_words = words[:-1] + [completed_word]
        generated_text = [completed_word]
        
        n = self.n_gram_model.n
        context = updated_words[-(n - 1):] if n > 1 else []

        # генерация продолжений текста
        for _ in range(n_words):
            next_candidates, next_probabilities = self.n_gram_model.get_next_words_and_probs(context)
            if not next_candidates:
                break
            
            best_word = next_candidates[next_probabilities.index(max(next_probabilities))]
            generated_text.append(best_word)
            
            # обновляем контекст
            context = context[1:] + [best_word] if n > 1 else [best_word]
        
        suggestions.append(generated_text)
        return suggestions

In [34]:
dummy_corpus = [
    ['aa', 'aa', 'aa', 'aa', 'ab'],
    ['aaa', 'abab'],
    ['abb', 'aa', 'ab', 'bba', 'bbb', 'bcd']
]

word_completor = WordCompletor(dummy_corpus)
n_gram_model = NGramLanguageModel(corpus=dummy_corpus, n=2)
text_suggestion = TextSuggestion(word_completor, n_gram_model)

assert text_suggestion.suggest_text(['aa', 'aa'], n_words=3, n_texts=1) == [['aa', 'aa', 'aa', 'aa']]
assert text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1) == [['ab', 'bba', 'bbb']]

In [35]:
# проверка результата выполнения
result = text_suggestion.suggest_text(['abb', 'aa', 'ab'], n_words=2, n_texts=1)
print(f"Generated suggestion: {result}")  # выводим результат

Generated suggestion: [['ab', 'bba', 'bbb']]


In [36]:
text_suggestion = TextSuggestion(word_completor, n_gram_model)

## Часть 2

Настало время довести вашу систему до ума. В этой части вы можете модифицировать все классы по своему усмотрению и добавлять любые эвристики. Если нужно, то дополнительно обрабатывать текст и вообще делать все, что считаете нужным, __кроме использования дополнительных данных__. Главное – вы должны обернуть вашу систему в пользовательский интерфейс с помощью [reflex](https://github.com/reflex-dev/reflex). В нем можно реализовать почти любой функционал по вашему желанию.

Мы настоятельно рекомендуем вам оформить код в проект, а не писать в ноутбуке. Но если вам очень хочется писать тут, то хотя бы не меняйте код в предыдущих заданиях, чтобы его можно было нормально оценивать.

При сдаче решения прикрепите весь ваш __код__, __отчет__ по второй части и __видео__ с демонстрацией работы вашей системы. Удачи!